<h1>Lab Assignment 1 </h1>
<h3>E-commerce platform chosen: Lazada</h3>
<h3>Product: Nestle Milo</h3>
<h5>Group member: <br>1. Vishnu Ram A/L Kartigesa Naidu SW01083727<br>2. Jeevesh A/L Perapakaran SW01083692<br>3. Thong Hao Hong SW01083725</h5>

In [1]:
#Import all of the required libraries, including selenium for web automation, and bs4 to parse the html content 
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup


In [2]:
#This function is used to resolve the slider captcha by using selenium action chain to instantiate a chain of predefined actions.
def verify_captcha(driver, wait):
    try:
        #Tell system to sleep for 5s.
        time.sleep(5)
        #Find the iframe where slider captcha resides.
        iframe = driver.execute_script('return document.querySelectorAll("iframe")[1]')
        print(f"Second captcha: {iframe} ")
        
        if iframe:
            #switch the driver from main document to captcha iframe
            WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR, "#baxia-dialog-content")))
            #assign the slider element to a variable
            slider = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "#nc_1_n1z"))
            )
            #Print the selected items in the console for debugging purposes
            driver.execute_script("console.log(arguments[0] + 'hi')", slider, iframe)
            print("Captcha slider found.", slider)
            #wait until the driver is clickable
            wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="nc_1_n1z"]')))
            #initialize action chain
            action = ActionChains(driver)
            #add the first action to the chain, hold the slider 
            action.click_and_hold(slider).perform()
            time.sleep(0.5) 
            #define the slider width
            distance = 248 
            current_position = 0
            #Create an infinite loop to pull the slider to a specific position until it reaches the end
            while current_position < distance:
                #define the step to be moved for this iteration
                step = random.randint(15, 40)  
                #move the slider according to the step
                action.move_by_offset(step, 0).perform()
                #compound current_position with current step
                current_position += step
                #tell system to sleep to mimick human behavior
                time.sleep(random.uniform(0.05, 0.15))  

            #release the slider once it exceeds the width
            action.release().perform()
            print("Captcha solved.")
    
            time.sleep(2)
            #change it back to initial document
            driver.switch_to.default_content()
    #catch exception
    except Exception as e:
        print(f"{e}")
        driver.switch_to.default_content()

In [3]:
#define human agents argument, then add it to the edge driver header
edge_options = Options()

#create webdriver
driver = webdriver.Edge(options=edge_options)
driver.delete_all_cookies()

#command driver to open lazada product webpage
driver.get("https://www.lazada.com.my/products/nestle-milo-chocolate-malt-powder-tin-14kg-i14299258-s17622746.html?scm=1007.17760.398138.0&pvid=84704c7a-eb29-476e-b395-a11d630b6db8&search=flashsale&spm=a2o4k.homepage.FlashSale.d_14299258")

reviews = {}

#define driver wait time
wait = WebDriverWait(driver, 10)

#If captcha occur upon landing, refresh the page to bypass the first captcha
try:
    wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'baxia-dialog-close')))
    driver.refresh()
    time.sleep(2)
except Exception as e:
    print("No captcha")

#click on the review button which will scroll automatically to the review pane section.
review_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="module_product_review_star_1"]/div/a[1]'))).click()
#Fetch 5 pages of review starting from page 1
for i in range(1,6):
        #find the div or container that comprises all of the review information
        reviews_container = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'mod-reviews')))
        #get the inner content of the review container above
        innerhtml =  driver.execute_script("return arguments[0].innerHTML;", reviews_container)
        #parse the innerhtml with bs and store it in the review dictionary along with the index
        reviews[i] = BeautifulSoup(innerhtml, "html.parser")
        
        time.sleep(2)
        #get the current pagination button
        current_button = driver.execute_script('return document.querySelector(".next-btn.next-btn-normal.current")')
        #based on the current button, find its next element and click
        driver.execute_script("console.log(arguments[0].nextElementSibling.textContent)", current_button)
        driver.execute_script("arguments[0].nextElementSibling.click()", current_button)
        time.sleep(5)
        #check whether second captcha is shown or no
        verify_captcha(driver, wait)
print(reviews)
driver.quit()

No captcha
Second captcha: None 
Second captcha: None 
Second captcha: None 
Second captcha: None 
Second captcha: None 
{1: <div class="item"><div class="top"><div class="container-star starCtn left" style="width: 83.125px; height: 16.625px;"><img class="star" src="//img.lazcdn.com/g/tps/tfs/TB19ZvEgfDH8KJjy1XcXXcpdXXa-64-64.png" style="width: 16.625px; height: 16.625px;"/><img class="star" src="//img.lazcdn.com/g/tps/tfs/TB19ZvEgfDH8KJjy1XcXXcpdXXa-64-64.png" style="width: 16.625px; height: 16.625px;"/><img class="star" src="//img.lazcdn.com/g/tps/tfs/TB19ZvEgfDH8KJjy1XcXXcpdXXa-64-64.png" style="width: 16.625px; height: 16.625px;"/><img class="star" src="//img.lazcdn.com/g/tps/tfs/TB19ZvEgfDH8KJjy1XcXXcpdXXa-64-64.png" style="width: 16.625px; height: 16.625px;"/><img class="star" src="//img.lazcdn.com/g/tps/tfs/TB19ZvEgfDH8KJjy1XcXXcpdXXa-64-64.png" style="width: 16.625px; height: 16.625px;"/></div><span class="title right">3 weeks ago</span></div><div class="middle"><span>L***a</sp

In [4]:
#define the structure of a dataframe
details = {'name':[], 'date':[], 'content':[]}
for i, j in reviews.items():
    #find all of the reviews in a page
    items = j.find_all(class_="item")
    for itemcontent in items:
        #retrieve the date, name and content based on class name then append it to the specific list store in the dictionary
        details['date'].append(itemcontent.find(class_ = 'title right').text)
        details['name'].append(itemcontent.select('.middle span')[0].text)
        details['content'].append(itemcontent.select('.item-content .content')[0].text)
print(details)

{'name': ['L***a', 'Lau M.', 'K***.', 'T***.', 'Rodney T.', 'P***.', 'Ong C.', 'S***.', 'H***.', 'G***.', 'S***.', 'J***a', 'Steven T.', 'K***h', 'L***g', 'shumaine C.', 'MeiYun L.', 'Emily L.', 'T***.', 'a***.', 'Siew W.', 'Tee B.', 'Chloe L.', 'H***.', 'L***.'], 'date': ['3 weeks ago', '1 week ago', '1 week ago', '2 weeks ago', '3 weeks ago', '1 week ago', '06 Feb 2025', '08 Feb 2025', '06 Feb 2025', '04 Feb 2025', '2 weeks ago', '1 week ago', '01 Feb 2025', '27 Jan 2025', '24 Jan 2025', '3 weeks ago', '06 Feb 2025', '09 Feb 2025', '23 Jan 2025', '2 weeks ago', '17 Dec 2024', '15 Dec 2024', '22 Jan 2025', '04 Dec 2024', '13 Dec 2024'], 'content': ['Convenient and easy to prepare, Great source of vitamins and minerals,', 'Taste:  traditional taste of milk Perfect for hot or cold weather, Perfect for breakfast, \nWhy reduce to 1.4kg now??', 'Great source of vitamins and minerals, Rich chocolate flavor, Ideal for sports and physical activities, expiry 13.06.2026.', 'Keeps me refreshed t

In [5]:
import pandas as pd
#convert the dictionary into DataFrame
data = pd.DataFrame(details, columns=details.keys())
data

,name,date,content
0,L***a,3 weeks ago,"Convenient and easy to prepare, Great source o..."
1,Lau M.,1 week ago,Taste: traditional taste of milk Perfect for ...
2,K***.,1 week ago,"Great source of vitamins and minerals, Rich ch..."
3,T***.,2 weeks ago,"Keeps me refreshed throughout the day, Great s..."
4,Rodney T.,3 weeks ago,good received today packaging good condition b...
5,P***.,1 week ago,"Super fast and safety delivery, a classic favo..."
6,Ong C.,06 Feb 2025,Good product quality and good value for money ...
7,S***.,08 Feb 2025,"Provides a boost of energy, Keeps me refreshed..."
8,H***.,06 Feb 2025,"Great source of vitamins and minerals, Conveni..."
9,G***.,04 Feb 2025,"Great source of vitamins and minerals, Perfect..."


In [6]:
#save it in csv format
data.to_csv("lazada_review.csv")